In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [4]:
# Load the data
filepath = r'C:\Users\Admin\OneDrive - Hanoi University of Science and Technology\Training đi thi\Project\Advanced_Regression\data\raw\train.csv'
data = pd.read_csv(filepath, index_col='Id')

# Fill the missing

## Numerical Features

In [5]:
data['LotFrontage'] = data['LotFrontage'].fillna(data[data['LotFrontage'] < 300]['LotFrontage'].mean())

data['GarageYrBlt'] = data['GarageYrBlt'].fillna(data['GarageYrBlt'].interpolate())
data['MasVnrArea'] = data['MasVnrArea'].fillna(0)

In [6]:
for col in ['LotFrontage', 'GarageYrBlt', 'MasVnrArea', 'MasVnrType']:
    print(data[col].isnull().sum())

0
0
0
872


## Categorical Features

In [7]:
data['MasVnrType'] = data['MasVnrType'].fillna('None')
data.drop(['MiscFeature','PoolQC','Fence','Alley'],axis=1, inplace=True)

In [8]:
lst_of_missing = []
for col in data.columns:
    if data[col].isnull().sum() > 0:
        lst_of_missing.append(col)
        print(f'{col:<13}: {data[col].isnull().sum(): <4} missing values - {data[col].isnull().sum() / len(data) * 100:.2f}% - {data[col].dtype}')

BsmtQual     : 37   missing values - 2.53% - object
BsmtCond     : 37   missing values - 2.53% - object
BsmtExposure : 38   missing values - 2.60% - object
BsmtFinType1 : 37   missing values - 2.53% - object
BsmtFinType2 : 38   missing values - 2.60% - object
Electrical   : 1    missing values - 0.07% - object
FireplaceQu  : 690  missing values - 47.26% - object
GarageType   : 81   missing values - 5.55% - object
GarageFinish : 81   missing values - 5.55% - object
GarageQual   : 81   missing values - 5.55% - object
GarageCond   : 81   missing values - 5.55% - object


In [9]:
print(data['SalePrice'].max())
print(data['SalePrice'].min())

755000
34900


In [10]:
def fill_missing(df, col):
    # Step 1: Set point value
    point = {}
    for type in df[col].unique():
        num = df[df[col] == type][col].count()
        if num > 0:
            point[type] = 1/num
        else:
            point[type] = 0
    # Step 2: Find mode value for each range
    mode = {}
    for i in range(5):
        rang = [i*144020, (i+1)*144020]
        data = df[col][df['SalePrice'].between(rang[0], rang[1])]
        max = 0
        for type in data.unique():
            if data[data == type].count()*point[type] > max:
                mode[i] = type
                max = data[data == type].count()*point[type]
    # Step 3: Fill missing value
    for i in df[df[col].isnull()].index:
        for c in mode.keys():
            if df['SalePrice'][i] in range(c*144020, (c+1)*144020):
                df[col][i] = mode[c]
            
fill_missing(data, 'Electrical')
print(data['Electrical'].isnull().sum())

for col in lst_of_missing:
    fill_missing(data, col)


0


C:\Users\Admin\AppData\Local\Temp\ipykernel_26644\2176639252.py:24: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df[col][i] = mode[c]
C:\Users\Admin\AppData\Local\Temp\ipykernel_26644\2176639252.py:24: SettingWithCopyWarning: 
A value is tr

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1460 entries, 1 to 1460
Data columns (total 76 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1460 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   Utilities      1460 non-null   object 
 8   LotConfig      1460 non-null   object 
 9   LandSlope      1460 non-null   object 
 10  Neighborhood   1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Condition2     1460 non-null   object 
 13  BldgType       1460 non-null   object 
 14  HouseStyle     1460 non-null   object 
 15  OverallQual    1460 non-null   int64  
 16  OverallCond    1460 non-null   int64  
 17  YearBuilt      1460 non-null   int64  
 18  YearRemodAdd 

# Encoding

In [12]:
label_encoders = LabelEncoder()
for col in data.columns:
    if data[col].dtype == 'object':
        data[col] = label_encoders.fit_transform(data[col])

In [13]:
data.select_dtypes(include=['object'])

""
Id
1
2
3
4
5
...
1456
1457
1458


# Normalization

## Use StandardScaler
- StandardScaler is used to scale the data to have a mean of 0 and a standard deviation of 1.

In [14]:
scaler = StandardScaler()
data_standardized = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
print(data_standardized.head())

   MSSubClass  MSZoning  LotFrontage   LotArea    Street  LotShape  \
0    0.073375 -0.045532    -0.226101 -0.207142  0.064238  0.750731   
1   -0.872563 -0.045532     0.455190 -0.091886  0.064238  0.750731   
2    0.073375 -0.045532    -0.089843  0.073480  0.064238 -1.378933   
3    0.309859 -0.045532    -0.453198 -0.096897  0.064238 -1.378933   
4    0.073375 -0.045532     0.636868  0.375148  0.064238 -1.378933   

   LandContour  Utilities  LotConfig  LandSlope  ...  EnclosedPorch  \
0     0.314667   -0.02618   0.604670  -0.225716  ...      -0.359325   
1     0.314667   -0.02618  -0.628316  -0.225716  ...      -0.359325   
2     0.314667   -0.02618   0.604670  -0.225716  ...      -0.359325   
3     0.314667   -0.02618  -1.861302  -0.225716  ...       4.092524   
4     0.314667   -0.02618  -0.628316  -0.225716  ...      -0.359325   

   3SsnPorch  ScreenPorch  PoolArea   MiscVal    MoSold    YrSold  SaleType  \
0  -0.116339    -0.270208 -0.068692 -0.087688 -1.599111  0.138777  0.3138